# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filetoload="../WeatherPy/output/cityweather.csv"
cities=pd.read_csv(filetoload)
cities.head()


,Unnamed: 0,City,Country,Lat,Lon,Tmax,% H,Cloudiness,Wind v,Date
0,0,Mayo,US,38.8876,-76.5119,46.00,100,90,3.44,1614607236
1,1,Clyde River,CA,70.4692,-68.5914,-14.80,64,75,24.16,1614607237
2,2,Albany,US,42.6001,-73.9662,39.99,86,100,6.62,1614607028
3,3,Concepción del Oro,MX,24.6333,-101.4167,54.14,41,23,1.50,1614607240
4,4,Cape Town,ZA,-33.9258,18.4232,84.20,34,40,8.05,1614607162


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity=cities["% H"].astype(float)
latlon=cities[["Lat","Lon"]].astype(float)
max_h=humidity.max()

In [4]:
fig=gmaps.figure(center=(0,0),zoom_level=2)
heat=gmaps.heatmap_layer(latlon,weights=humidity,dissipating=False,max_intensity=max_h,point_radius=2)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealrun=cities.loc[(cities['Tmax']>70) & 
                    (cities['Tmax']<100) &
                    (cities['Wind v']>0) &
                    (cities['Wind v']<10) &
                    (cities['Cloudiness']==0)]
#idealrun.shape
idealrun.reset_index(inplace=True)
del idealrun['index']
idealrun.head(20)

,Unnamed: 0,City,Country,Lat,Lon,Tmax,% H,Cloudiness,Wind v,Date
0,48,Jalu,LY,29.0331,21.5482,70.95,28,0,7.67,1614607299
1,63,Puerto Ayora,EC,-0.7393,-90.3518,75.99,98,0,5.99,1614607320
2,64,Eravur,LK,7.7733,81.6095,78.17,67,0,5.39,1614607321
3,65,Lüderitz,NaN,-26.6481,15.1594,72.77,64,0,6.96,1614607323
4,148,Constitución,CL,-35.3333,-72.4167,70.38,55,0,2.71,1614607299
5,151,Itimādpur,IN,27.2500,78.2000,72.70,20,0,9.37,1614607437
6,193,Glencoe,ZA,-28.1783,30.1470,73.76,38,0,6.78,1614607496
7,224,Kātrās,IN,23.8000,86.2833,82.06,18,0,7.11,1614607537
8,233,Adrar,MR,20.5022,-10.0711,78.01,17,0,8.39,1614607551
9,238,Port Blair,IN,11.6667,92.7500,79.50,65,0,7.63,1614607559


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels=[]
for i in range(len(idealrun)):
    Lat=idealrun.loc[i]['Lat']
    Lon=idealrun.loc[i]['Lon']
    parametros={'Location':f'{Lat},{Lon}',
                'radius':5000,
                'types':'hotel',
                'key':g_key}

    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request=requests.get(base_url,params=parametros)
    rjson=request.json()
    try:
        hotels.append(rjson['results'][0]['name'])
    except:
        hotels.append('')
idealrun['Hotel name']=hotels
idealrun

,Unnamed: 0,City,Country,Lat,Lon,Tmax,% H,Cloudiness,Wind v,Date,Hotel name
0,48,Jalu,LY,29.0331,21.5482,70.95,28,0,7.67,1614607299,
1,63,Puerto Ayora,EC,-0.7393,-90.3518,75.99,98,0,5.99,1614607320,
2,64,Eravur,LK,7.7733,81.6095,78.17,67,0,5.39,1614607321,
3,65,Lüderitz,NaN,-26.6481,15.1594,72.77,64,0,6.96,1614607323,
4,148,Constitución,CL,-35.3333,-72.4167,70.38,55,0,2.71,1614607299,
5,151,Itimādpur,IN,27.2500,78.2000,72.70,20,0,9.37,1614607437,
6,193,Glencoe,ZA,-28.1783,30.1470,73.76,38,0,6.78,1614607496,
7,224,Kātrās,IN,23.8000,86.2833,82.06,18,0,7.11,1614607537,
8,233,Adrar,MR,20.5022,-10.0711,78.01,17,0,8.39,1614607551,
9,238,Port Blair,IN,11.6667,92.7500,79.50,65,0,7.63,1614607559,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels= [info_box_template.format(**row) for index, row in idealrun.iterrows()]
locations = idealrun[["Lat", "Lon"]]

In [8]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))